### Based on https://developer.hashicorp.com/vault/tutorials/kubernetes/kubernetes-minikube-tls

In [ ]:
%%bash
minikube delete -p dc1

In [ ]:
%%bash
# Initialize docker first
#sleep 30
minikube start -p dc1

In [ ]:
%%bash
minikube status -p dc1

In [ ]:
%%bash
helm repo add hashicorp https://helm.releases.hashicorp.com
helm repo update

In [306]:
%%bash
rm -rf /tmp/vault
mkdir /tmp/vault

In [307]:
%%bash
export VAULT_K8S_NAMESPACE="vault" \
export VAULT_HELM_RELEASE_NAME="vault" \
export VAULT_SERVICE_NAME="vault-internal" \
export K8S_CLUSTER_NAME="cluster.local" \
export WORKDIR=/tmp/vault

openssl genrsa -out ${WORKDIR}/vault.key 2048
cat > ${WORKDIR}/vault-csr.conf <<EOF
[req]
default_bits = 2048
prompt = no
encrypt_key = yes
default_md = sha256
distinguished_name = kubelet_serving
req_extensions = v3_req
[ kubelet_serving ]
O = system:nodes
CN = system:node:*.${VAULT_HELM_RELEASE_NAME}.svc.${K8S_CLUSTER_NAME}
[ v3_req ]
basicConstraints = CA:FALSE
keyUsage = nonRepudiation, digitalSignature, keyEncipherment, dataEncipherment
extendedKeyUsage = serverAuth, clientAuth
subjectAltName = @alt_names
[alt_names]
DNS.1 = *.${VAULT_SERVICE_NAME}
DNS.2 = *.${VAULT_SERVICE_NAME}.${VAULT_HELM_RELEASE_NAME}.svc.${K8S_CLUSTER_NAME}
DNS.3 = *.${VAULT_HELM_RELEASE_NAME}
DNS.4 = *.${VAULT_HELM_RELEASE_NAME}.svc.${K8S_CLUSTER_NAME}
IP.1 = 127.0.0.1
EOF

openssl req -new -key ${WORKDIR}/vault.key -out ${WORKDIR}/vault.csr -config ${WORKDIR}/vault-csr.conf


cat > ${WORKDIR}/csr.yaml <<EOF
apiVersion: certificates.k8s.io/v1
kind: CertificateSigningRequest
metadata:
   name: vault.svc
spec:
   signerName: kubernetes.io/kubelet-serving
   expirationSeconds: 8640000
   request: $(cat ${WORKDIR}/vault.csr|base64|tr -d '\n')
   usages:
   - digital signature
   - key encipherment
   - server auth
EOF

kubectl create -f ${WORKDIR}/csr.yaml


Generating RSA private key, 2048 bit long modulus
....+++++
..................................................+++++
e is 65537 (0x10001)


certificatesigningrequest.certificates.k8s.io/vault.svc created


In [308]:
%%bash
kubectl certificate approve vault.svc

certificatesigningrequest.certificates.k8s.io/vault.svc approved


In [309]:
%%bash
kubectl get csr vault.svc

NAME        AGE   SIGNERNAME                      REQUESTOR       REQUESTEDDURATION   CONDITION
vault.svc   5s    kubernetes.io/kubelet-serving   minikube-user   100d                Approved,Issued


In [310]:
%env WORKDIR=/tmp/vault
%env VAULT_K8S_NAMESPACE=vault
%env VAULT_HELM_RELEASE_NAME=vault
%env VAULT_SERVICE_NAME=vault-internal 
%env K8S_CLUSTER_NAME=cluster.local 

env: WORKDIR=/tmp/vault
env: VAULT_K8S_NAMESPACE=vault
env: VAULT_HELM_RELEASE_NAME=vault
env: VAULT_SERVICE_NAME=vault-internal
env: K8S_CLUSTER_NAME=cluster.local


In [311]:
%%bash
kubectl get csr vault.svc -o jsonpath='{.status.certificate}' | openssl base64 -d -A -out ${WORKDIR}/vault.crt
kubectl config view \
--raw \
--minify \
--flatten \
-o jsonpath='{.clusters[].cluster.certificate-authority-data}' \
| base64 -d > ${WORKDIR}/vault.ca

kubectl create namespace $VAULT_K8S_NAMESPACE

kubectl create secret generic vault-ha-tls \
   -n $VAULT_K8S_NAMESPACE \
   --from-file=vault.key=${WORKDIR}/vault.key \
   --from-file=vault.crt=${WORKDIR}/vault.crt \
   --from-file=vault.ca=${WORKDIR}/vault.ca


namespace/vault created
secret/vault-ha-tls created


### Adding enterprise license based on https://developer.hashicorp.com/vault/docs/platform/k8s/helm/enterprise

In [312]:
%%bash 
secret=$(cat /Users/jose/Demo/Vault_ADP/ADP_PlayGround/vault.hclic)
kubectl create secret generic vault-ent-license --from-literal="license=${secret}" -n $VAULT_K8S_NAMESPACE


secret/vault-ent-license created


In [313]:
%%bash
kubectl apply -f config-map-logrotate.yaml -n vault

configmap/logrotate-config created


In [314]:
%%bash
cat > ${WORKDIR}/overrides.yaml <<EOF
global:
   enabled: true
   tlsDisable: false # Disabling TLS to avoid issues when connecting to Vault via port forwarding
injector:
   enabled: true
server:
# config.yaml
   image:
      repository: hashicorp/vault-enterprise
      tag: 1.14.0-ent
   enterpriseLicense:
      secretName: vault-ent-license

   extraEnvironmentVars:
      VAULT_CACERT: /vault/userconfig/vault-ha-tls/vault.ca
      VAULT_TLSCERT: /vault/userconfig/vault-ha-tls/vault.crt
      VAULT_TLSKEY: /vault/userconfig/vault-ha-tls/vault.key
   volumes:
      - name: userconfig-vault-ha-tls
        secret:
         defaultMode: 420
         secretName: vault-ha-tls
      - name: logrotate-config
        configMap:
          name: logrotate-config
   volumeMounts:
      - mountPath: /vault/userconfig/vault-ha-tls
        name: userconfig-vault-ha-tls
        readOnly: true
   standalone:
      enabled: false
   
   affinity: ""
   auditStorage:
      enabled: true

   ha:
      enabled: true
      replicas: 3
      raft:
         enabled: true
         setNodeId: true
         config: |
            ui = true
            listener "tcp" {
               tls_disable = 0 # Disabling TLS to avoid issues when connecting to Vault via port forwarding
               address = "[::]:8200"
               cluster_address = "[::]:8201"
               tls_cert_file = "/vault/userconfig/vault-ha-tls/vault.crt"
               tls_key_file  = "/vault/userconfig/vault-ha-tls/vault.key"
               tls_client_ca_file = "/vault/userconfig/vault-ha-tls/vault.ca"
               # api_address = "https://vault-active.vault.svc.cluster.local:8200"
            }
            storage "raft" {
               path = "/vault/data"
            
               retry_join {
                  auto_join             = "provider=k8s namespace=vault label_selector=\"component=server,app.kubernetes.io/name=vault\""
                  auto_join_scheme      = "https"
                  leader_ca_cert_file   = "/vault/userconfig/vault-ha-tls/vault.ca"
                  leader_tls_servername = "vault-0.vault-internal" #Tiene que matchear una SAN del certificado
               }
            
            }
            disable_mlock = true
            service_registration "kubernetes" {}
   
   # HUP signal for logrotate
   shareProcessNamespace: true
 
   # And finally the container
   extraContainers:
     - name: auditlog-rotator
       image: josemerchan/vault-logrotate:0.0.1
       imagePullPolicy: Always
       env:
         - name: CRONTAB
           value: "*/5 * * * *"
       volumeMounts:
         - mountPath: /etc/logrotate.conf
           name: logrotate-config
           subPath: logrotate.conf
           readOnly: true
         - mountPath: /vault/audit
           name: audit
EOF


In [315]:
%%bash
helm install -n $VAULT_K8S_NAMESPACE $VAULT_HELM_RELEASE_NAME hashicorp/vault -f ${WORKDIR}/overrides.yaml

NAME: vault
LAST DEPLOYED: Fri Jul 21 16:16:59 2023
NAMESPACE: vault
STATUS: deployed
REVISION: 1
NOTES:
Thank you for installing HashiCorp Vault!

Now that you have deployed Vault, you should look over the docs on using
Vault with Kubernetes available here:

https://www.vaultproject.io/docs/


Your release is named vault. To learn more about the release, try:

  $ helm status vault
  $ helm get manifest vault


In [318]:
%%bash
kubectl -n $VAULT_K8S_NAMESPACE get pods
kubectl get events -n $VAULT_K8S_NAMESPACE --sort-by='.metadata.creationTimestamp'

NAME                                    READY   STATUS    RESTARTS   AGE
vault-0                                 1/2     Running   0          32s
vault-1                                 1/2     Running   0          32s
vault-2                                 1/2     Running   0          32s
vault-agent-injector-6fdf6cd4d6-lk78m   1/1     Running   0          33s
LAST SEEN   TYPE      REASON                  OBJECT                                       MESSAGE
33s         Normal    SuccessfulCreate        statefulset/vault                            create Claim data-vault-0 Pod vault-0 in StatefulSet vault success
33s         Normal    NoPods                  poddisruptionbudget/vault                    No matching pods found
33s         Normal    Scheduled               pod/vault-agent-injector-6fdf6cd4d6-lk78m    Successfully assigned vault/vault-agent-injector-6fdf6cd4d6-lk78m to dc1
33s         Normal    ScalingReplicaSet       deployment/vault-agent-injector              Scaled up

In [319]:
%%bash
kubectl exec -n $VAULT_K8S_NAMESPACE vault-0 -c vault -- vault operator init \
    -key-shares=1 \
    -key-threshold=1 \
    -format=json > ${WORKDIR}/cluster-keys.json


In [320]:
%%bash
jq -r ".unseal_keys_b64[]" ${WORKDIR}/cluster-keys.json
VAULT_UNSEAL_KEY=$(jq -r ".unseal_keys_b64[]" ${WORKDIR}/cluster-keys.json)

z05OcdEEQW1x3PAkOclOb0oxItxAbrxf24he5IZDoPE=


## Initialize Containers

In [321]:
%%bash
kubectl exec -n $VAULT_K8S_NAMESPACE vault-0 -c vault -- vault operator unseal $(jq -r ".unseal_keys_b64[]" ${WORKDIR}/cluster-keys.json)

Key                     Value
---                     -----
Seal Type               shamir
Initialized             true
Sealed                  false
Total Shares            1
Threshold               1
Version                 1.14.0+ent
Build Date              2023-06-19T16:20:54Z
Storage Type            raft
Cluster Name            vault-cluster-6446f817
Cluster ID              47041f01-c10b-d262-462d-23f7ede70441
HA Enabled              true
HA Cluster              https://vault-0.vault-internal:8201
HA Mode                 active
Active Since            2023-07-21T14:18:02.691862836Z
Raft Committed Index    79
Raft Applied Index      79
Last WAL                32


In [322]:
%%bash
kubectl exec -n $VAULT_K8S_NAMESPACE -ti vault-1 -c vault -- vault operator unseal $(jq -r ".unseal_keys_b64[]" ${WORKDIR}/cluster-keys.json)

Unable to use a TTY - input is not a terminal or the right kind of file


Key                Value
---                -----
Seal Type          shamir
Initialized        true
Sealed             true
Total Shares       1
Threshold          1
Unseal Progress    0/1
Unseal Nonce       n/a
Version            1.14.0+ent
Build Date         2023-06-19T16:20:54Z
Storage Type       raft
HA Enabled         true


In [323]:
%%bash
kubectl exec -n $VAULT_K8S_NAMESPACE -ti vault-2 -c vault -- vault operator unseal $(jq -r ".unseal_keys_b64[]" ${WORKDIR}/cluster-keys.json)

Unable to use a TTY - input is not a terminal or the right kind of file


Key                Value
---                -----
Seal Type          shamir
Initialized        true
Sealed             true
Total Shares       1
Threshold          1
Unseal Progress    0/1
Unseal Nonce       n/a
Version            1.14.0+ent
Build Date         2023-06-19T16:20:54Z
Storage Type       raft
HA Enabled         true


In [324]:
%%bash
kubectl exec -n $VAULT_K8S_NAMESPACE -ti vault-0 -c vault -- vault status

Unable to use a TTY - input is not a terminal or the right kind of file


Key                     Value
---                     -----
Seal Type               shamir
Initialized             true
Sealed                  false
Total Shares            1
Threshold               1
Version                 1.14.0+ent
Build Date              2023-06-19T16:20:54Z
Storage Type            raft
Cluster Name            vault-cluster-6446f817
Cluster ID              47041f01-c10b-d262-462d-23f7ede70441
HA Enabled              true
HA Cluster              https://vault-0.vault-internal:8201
HA Mode                 active
Active Since            2023-07-21T14:18:02.691862836Z
Raft Committed Index    99
Raft Applied Index      99
Last WAL                35


In [325]:
%%bash
kubectl exec -n $VAULT_K8S_NAMESPACE -ti vault-1 -c vault -- vault status

Unable to use a TTY - input is not a terminal or the right kind of file


Key                                    Value
---                                    -----
Seal Type                              shamir
Initialized                            true
Sealed                                 false
Total Shares                           1
Threshold                              1
Version                                1.14.0+ent
Build Date                             2023-06-19T16:20:54Z
Storage Type                           raft
Cluster Name                           vault-cluster-6446f817
Cluster ID                             47041f01-c10b-d262-462d-23f7ede70441
HA Enabled                             true
HA Cluster                             https://vault-0.vault-internal:8201
HA Mode                                standby
Active Node Address                    https://10.244.0.67:8200
Performance Standby Node               true
Performance Standby Last Remote WAL    35
Raft Committed Index                   101
Raft Applied Index                     101

In [326]:
%%bash
kubectl exec -n $VAULT_K8S_NAMESPACE -ti vault-2 -c vault -- vault status

Unable to use a TTY - input is not a terminal or the right kind of file


Key                                    Value
---                                    -----
Seal Type                              shamir
Initialized                            true
Sealed                                 false
Total Shares                           1
Threshold                              1
Version                                1.14.0+ent
Build Date                             2023-06-19T16:20:54Z
Storage Type                           raft
Cluster Name                           vault-cluster-6446f817
Cluster ID                             47041f01-c10b-d262-462d-23f7ede70441
HA Enabled                             true
HA Cluster                             https://vault-0.vault-internal:8201
HA Mode                                standby
Active Node Address                    https://10.244.0.67:8200
Performance Standby Node               true
Performance Standby Last Remote WAL    35
Raft Committed Index                   103
Raft Applied Index                     103

## Get Root Token and port forward to Vault on port 8200

In [327]:
%%bash
cat ${WORKDIR}/cluster-keys.json | jq -r ".root_token"

hvs.DHb7N6blLL0vg9NeZDfaKAwz


In [329]:
%%bash
kubectl exec -ti vault-0 -n vault -c vault --  vault login hvs.DHb7N6blLL0vg9NeZDfaKAwz

Unable to use a TTY - input is not a terminal or the right kind of file


Success! You are now authenticated. The token information displayed below
is already stored in the token helper. You do NOT need to run "vault login"
again. Future Vault requests will automatically use this token.

Key                  Value
---                  -----
token                hvs.DHb7N6blLL0vg9NeZDfaKAwz
token_accessor       qS5UAnmwQaPgmv8eoQ0WKFaz
token_duration       ∞
token_renewable      false
token_policies       ["root"]
identity_policies    []
policies             ["root"]


In [330]:
%%bash
kubectl exec -ti vault-0 -n vault -c vault --  vault audit enable file file_path=/vault/audit/vault.log

Unable to use a TTY - input is not a terminal or the right kind of file


Success! Enabled the file audit device at: file/


In [331]:
%%bash
kubectl exec -ti vault-0 -n vault -c vault --  cat /vault/audit/vault_audit.log

Unable to use a TTY - input is not a terminal or the right kind of file


{"time":"2023-07-21T14:18:37.481134422Z","type":"request","auth":{"token_type":"default"},"request":{"id":"68b6d614-a641-b9d2-58a7-bd13f2d95b1c","operation":"update","namespace":{"id":"root"},"path":"sys/audit/test"}}
{"time":"2023-07-21T14:18:37.489565005Z","type":"response","auth":{"client_token":"hmac-sha256:a470da8845279c71f5c32f5429d3c2a6d35e968a87252324ea04ca0e13d9d0d7","accessor":"hmac-sha256:e9c8a7e997b2761cd5798b574fe4ee4565ec8e4bc44fcf899f91d2f9832d4644","display_name":"root","policies":["root"],"token_policies":["root"],"policy_results":{"allowed":true,"granting_policies":[{"name":"root","namespace_id":"root","type":"acl"}]},"token_type":"service","token_issue_time":"2023-07-21T14:17:53Z"},"request":{"id":"bdca6df9-580f-20bc-a1c8-cbebdd135937","client_id":"0DHqvq2D77kL2/JTPSZkTMJbkFVmUu0TzMi0jiXcFy8=","operation":"update","mount_point":"sys/","mount_type":"system","mount_accessor":"system_3efc35b5","mount_running_version":"v1.14.0+builtin.vault","mount_class":"secret","clien

In [339]:
%%bash
# Size in MB
kubectl exec -ti vault-0 -n vault -c vault --  ls -lh /vault/audit/vault_audit.log

Unable to use a TTY - input is not a terminal or the right kind of file


-rw-------    1 vault    vault       4.8M Jul 21 14:22 /vault/audit/vault_audit.log


In [ ]:
%%bash
#kubectl -n $VAULT_K8S_NAMESPACE get service vault
kubectl -n vault port-forward service/vault 8200:8200